### CR with Spacy - Example

In [15]:
import json, logging, sys
import pandas as pd

In [16]:
ner_file = "se_data/se_ner_annotated.tsv"
df_data = pd.read_csv(ner_file,sep="\t",encoding="latin1").fillna(method='ffill')

In [19]:
df_data = df_data[['Word', 'Tag']]
df_data.to_csv('spacy_ner.tsv', sep='\t',index=False)

In [20]:
# Convert .tsv file to dataturks json format. 

def tsv_to_json_format(input_path,output_path,unknown_label):
    
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("spacy_ner.tsv",'se_data/se_ner_spacy.json','abc')

In [22]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

#@plac.annotations(input_file=("Input file", "option", "i", str), output_file=("Output file", "option", "o", str))

def to_spacy_format(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None

to_spacy_format('se_data/se_ner_spacy.json', 'se_data/se_ner_spacy_new.json')


[('Word The elements include all hardware , software , equipment , facilities , personnel , processes , and procedures needed for this purpose ; that is , all things required to produce system level results ', {'entities': [(0, 4, 'Tag'), (30, 38, 'B-syscon'), (41, 49, 'B-syscon'), (52, 61, 'B-syscon'), (64, 74, 'B-syscon'), (77, 86, 'B-grp'), (89, 98, 'B-seterm'), (105, 115, 'B-opcon'), (183, 189, 'B-opcon'), (190, 195, 'I-opcon'), (196, 203, 'I-opcon')]}), ('The art is in knowing when and where to probe ', {'entities': []}), ('If the project waits until verification to do any type of test or analysis , any problems found will have a significant cost impact to redesign and reverify ', {'entities': [(7, 14, 'B-seterm'), (27, 39, 'B-opcon'), (58, 62, 'B-opcon'), (66, 74, 'B-opcon'), (135, 143, 'B-opcon'), (120, 124, 'B-mea'), (125, 131, 'I-mea'), (144, 147, 'I-opcon'), (148, 156, 'I-opcon')]}), ('For example , a trade study might be made to decide between performing a test at a centrali

In [ ]:
import spacy
nlp = spacy.blank('en')  # create blank Language class
ner = nlp.create_pipe('ner')

In [26]:
list(df_data.Tag.unique())

['O',
 'B-syscon',
 'B-grp',
 'B-seterm',
 'B-opcon',
 'I-opcon',
 'B-mea',
 'I-mea',
 'B-loc',
 'I-loc',
 'B-abb',
 'I-grp',
 'I-syscon',
 'B-cardinal',
 'B-org',
 'I-org',
 'B-event',
 'I-event',
 'I-seterm',
 'I-abb',
 'B-art']

In [36]:
#!/usr/bin/env python
# coding: utf8

# Training additional entity types using spaCy
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# New entity labels
# Specify the new entity labels which you want to add here
LABEL = list(df_data.Tag.unique())

# Loading training data 
with open ('se_data/se_ner_spacy_new.json', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

FULL_DATA = TRAIN_DATA
num_of_examples = len(FULL_DATA)
print(num_of_examples)

3606


In [37]:
TEST_DATA = FULL_DATA[int(num_of_examples*0.8):]
TRAIN_DATA = FULL_DATA[:int(num_of_examples*0.8)]

In [38]:
def train_2(model=None, new_model_name='animal', output_dir="se_data", n_iter=20):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    """ADD MULTIPLE LABELS TO NER MODEL"""
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                           losses=losses)
            print(losses)
            
    # test the trained model
    test_text = 'horses? fish?'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for i, ent in enumerate(doc.ents):
        print("Entity number %s is %s in text: '%s'" % (i, ent.label_, ent.text))

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [39]:
train_2()

Created blank 'en' model
{'ner': 9000.94977423923}
{'ner': 5355.5710320548105}
{'ner': 4530.165727838374}
{'ner': 4149.920792426277}
{'ner': 3689.099386887182}
{'ner': 3525.1057087612307}
{'ner': 3374.091743715449}
{'ner': 3048.4288279658163}
{'ner': 2900.2031110928156}
{'ner': 2817.6987098935515}
{'ner': 2669.930657240779}
{'ner': 2559.7106188265584}
{'ner': 2482.9755052962964}
{'ner': 2296.109947675886}
{'ner': 2260.6670425611205}
{'ner': 2221.8383112661386}
{'ner': 2092.8688851646375}
{'ner': 2061.536972688974}
{'ner': 1955.1068469380723}
{'ner': 1934.514745658943}
Entities in 'horses? fish?'
Saved model to se_data
Loading from se_data


In [42]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

examples = [
    ('Who is Shaka Khan?',
     [(7, 17, 'PERSON')]),
    ('I like London and Berlin.',
     [(7, 13, 'LOC'), (18, 24, 'LOC')])
]

ner_model = spacy.load('se_data') # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, TEST_DATA)
print(results)

{'uas': 0.0, 'las': 0.0, 'ents_p': 86.87196110210697, 'ents_r': 90.78590785907859, 'ents_f': 88.78582077190659, 'ents_per_type': {'B-seterm': {'p': 89.41605839416059, 'r': 96.45669291338582, 'f': 92.80303030303031}, 'B-mea': {'p': 92.9245283018868, 'r': 95.63106796116504, 'f': 94.25837320574162}, 'B-opcon': {'p': 88.06343906510851, 'r': 89.94032395566921, 'f': 88.991986503585}, 'I-opcon': {'p': 78.90818858560795, 'r': 84.3501326259947, 'f': 81.53846153846153}, 'B-syscon': {'p': 83.25688073394495, 'r': 92.8388746803069, 'f': 87.78718258766625}, 'B-cardinal': {'p': 100.0, 'r': 98.07692307692307, 'f': 99.02912621359222}, 'B-event': {'p': 95.08196721311475, 'r': 95.08196721311475, 'f': 95.08196721311475}, 'I-event': {'p': 87.17948717948718, 'r': 82.92682926829268, 'f': 85.00000000000001}, 'B-grp': {'p': 91.26984126984127, 'r': 95.0413223140496, 'f': 93.11740890688259}, 'I-seterm': {'p': 100.0, 'r': 75.0, 'f': 85.71428571428571}, 'B-org': {'p': 88.88888888888889, 'r': 80.0, 'f': 84.21052631

### References: 

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
https://timkuhn.github.io/TextMining/spacy/ner/2018/01/24/spaCy_NER_Training.html